In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd
import os
matplotlib.use('Qt5Agg')
%matplotlib qt5

In [2]:
outDirs = []
for root, dirs, files in os.walk('.'):
    if 'stab_ee'in root and 'outputs' in root:
        outDirs.append(root)
        
for i, Dir in enumerate(outDirs):
    print(i,Dir)

0 ./h_ethylene/stab_ee/10z_DZ+2p+keywords/g-2.5/outputs
1 ./h_ethylene/stab_ee/10z_DZ+2p+keywords/g-3.5/outputs
2 ./h_ethylene/stab_ee/3z_DZ+2p+keywords/g-2.5/outputs
3 ./h_ethylene/stab_ee/3z_DZ+2p+keywords/g-3.5/outputs
4 ./h_ethylene/stab_ee/3z_DZ+2p+keywords/g-3.5/outputs/.ipynb_checkpoints
5 ./oh_ethylene/stab_ee/optz_DZ+2p+keywords_B2/g-2.5/outputs
6 ./oh_ethylene/stab_ee/optz_DZ+2p+keywords_B2/outputs
7 ./oh_ethylene/stab_ee/10z_DZ+2p+keywords_B2/outputs
8 ./oh_ethylene/stab_ee/other_symms/optz_DZ+2p+keywords_B1/outputs
9 ./oh_ethylene/stab_ee/other_symms/10z_DZ+2p+keywords_A1/outputs
10 ./oh_ethylene/stab_ee/other_symms/optz_TZ+2p+keywords_A2/outputs
11 ./oh_ethylene/stab_ee/other_symms/optz_DZ+2p+keywords_A2/outputs
12 ./oh_ethylene/stab_ee/5z_DZ+2p+keywords_B2/outputs
13 ./oh_ethylene/stab_ee/optz_DZ+1p+keywords_B2/outputs
14 ./oh_ethylene/stab_ee/optz_DZ+4p+keywords_B2/outputs
15 ./oh_ethylene/stab_ee/optz_DZ+3p+keywords_B2/outputs


In [3]:
def is_state_naive(pattern, M_block, tol=0.1):
    truths=[]
    
    for i in pattern: # iterate over all atoms in the pattern
        q = float(M_block[i[0]-1].split()[5]) #del q (preserves sign)
        if i[1]-tol <= q <= i[1]+tol:         #does this work correctly for pos and neg numbers? yes.
            truths.append(bool(True))    #return true if atom n fits patterns
        else:
            truths.append(bool(False))   #or not
    
    
    if all(truths): #if every atom fits pattern, return true
        return True
    else:
        return False


def readfile(filename):
    f = open(filename, 'r')
    lines = f.readlines()
    f.close()
    return lines

# Naive $$q-tol\le q_{0} \le q+tol$$

In [4]:
#debug section
E2q = 33 #lines from Excitation energy line to first atom in Mulliken pop
tol = 0.1


#choose these, section
path = outDirs[6] #outfiles
atoms = 8 # number of atoms in molecule counting from 1
pattern = [(1,-0.5),(2,-0.5),(7,0.88),(8,0.12)] #[(atom #,charge)]


pat_alps = []
pat_Es = []

files = os.listdir(path)
files.sort()
print(path)

#does sts fit tolerance: yes/no
for file in files:
    lines = readfile(os.path.join(path,file))
    for i, line in enumerate(lines):
        if 'Excitation energy:' in line:
            #print(lines[i+E2q:i+E2q+atoms])
            if is_state_naive(pattern,lines[i+E2q:i+E2q+atoms],tol): # if Mulliken block matches pattern
                pat_alps.append(float(file[6:]))
                pat_Es.append(float(line.split()[2]))


# now plot it
data = pd.read_csv(path[:-8]+'/stab_data.csv', delim_whitespace = True)
print(np.shape(data))
print(data.head())

alps = data['z'].values
E_labels=data.columns[2:]
plt.cla()
plt.xlabel('alp')
plt.ylabel('E')
for E in E_labels:
    plt.plot(alps, data[E].values, '.')
plt.plot(pat_alps, pat_Es, 'ro')
#lt.ylim(0,7)
plt.show()

./oh_ethylene/stab_ee/optz_DZ+2p+keywords_B2/outputs
(41, 12)
        L         z         0         1         2         3         4  \
0  2.4694  0.636364  1.348699  1.749591  1.930578  2.820829  3.546807   
1  2.1704  0.678788  1.410712  1.833634  2.000029  2.903365  3.634412   
2  1.9225  0.721212  1.472543  1.915825  2.064399  2.983204  3.713133   
3  1.7149  0.763636  1.534401  1.996109  2.124044  3.060031  3.784987   
4  1.5391  0.806061  1.596478  2.074415  2.179342  3.133488  3.851631   

          5         6         7         8         9  
0  3.795749  4.680245  5.507099  5.596592  5.641564  
1  3.985649  4.719501  5.544042  5.669577  5.758070  
2  4.167909  4.753696  5.576807  5.741452  5.883771  
3  4.337214  4.791382  5.605610  5.812443  6.015826  
4  4.480365  4.848552  5.630682  5.882656  6.152384  


# Other Stuff
* sum dist
* avg dist
* ratio

In [6]:
def state_fit(pattern, M_block):
    
    fit = 0
    dist = []
    for i in pattern: # iterate over all atoms in the pattern
        q = float(M_block[i[0]-1].split()[5]) #del q (preserves sign)
        #print(i,q)
        
        #dist sum
        # fit += abs(i[1]-q)
        
        #dist avg
        dist.append(abs(i[1]-q))
    fit = sum(dist)/len(dist)
    
        #dist ratio sum thing
    #     Sum += q/i[1]
    # fit = Sum/len(Sum)
        
    return round(fit,6)


#debug section
E2q = 33 #lines from Excitation energy line to first atom in Mulliken pop
tol = 0.15


#choose these, section
path = outDirs[6] #outfiles
atoms = 8 # number of atoms in molecule counting from 1
pattern = [(1,-0.348512),(2,-0.348512),(7,0.886047),(8,0.087252)] #[(atom #,charge)]
files = os.listdir(path)
files.sort()
files

fit_alps = []
fit_es = []
fit_fit = []
#fit_plot([alps],[Es],[fit])
for i,file in enumerate(files):
    lines = readfile(os.path.join(path,file))
    for j, line in enumerate(lines):
        if 'Excitation energy:' in line:
            fit_alps.append(float(file[6:]))
            fit_es.append(float(lines[j].split()[2]))
            fit_fit.append(state_fit(pattern,lines[j+E2q:j+E2q+atoms]))

plt.cla()
plt.xlabel('alp')
plt.ylabel('E')
plt.scatter(fit_alps, fit_es, c=fit_fit, cmap=cm.Greys_r)
plt.colorbar()
plt.show()